## L1000FWD Predicted Side Effects Drug-Set Library
### Drug-set labels: Predicted Side Effects
#### ALL DATABASES ACCESSED 09/2019
##### Author : Eryk Kropiwnicki | eryk.kropiwnicki@icahn.mssm.edu

In [1]:
import json
import pandas as pd
import requests
import time
from collections import defaultdict
import csv
import os

In [2]:
os.chdir('../scripts')
from export_script import *
os.chdir('../L1000FWD')

In [3]:
df = pd.read_csv('input/pertid_to_name.tsv', delimiter = '\t')
df.head()

,pert_id,drug_name
0,BRD-A00100033,nifurtimox
1,BRD-A00520476,otenzepad
2,BRD-A00546892,biperiden
3,BRD-A00758722,noretynodrel
4,BRD-A00993607,alprenolol


In [4]:
pert_ids = df['pert_id'].tolist()
drug_name_conversion = df.set_index('pert_id').to_dict()['drug_name']

In [5]:
len(pert_ids)

4447

In [6]:
# Matching signatures to predicted side effects using SEP-L1000 API #
side_effect_dict = {}
failed_list = []

for pert_id in pert_ids:
    side_effect_list = []
    try:
        url = 'http://maayanlab.net/SEP-L1000/get_drug_se.php?pert_id='+pert_id
        response = requests.get(url)
    except ValueError:
        pass
    try:
        for item in response.json():
            side_effect_list.append((item['name']))
    except:
        failed_list.append(pert_id)
    side_effect_dict[pert_id] = side_effect_list
    time.sleep(0.5)

In [7]:
len(side_effect_dict)

4447

In [8]:
# Transposing side effect dictionary so that side effects are set-labels
# This function also throws out sets with len < 5
drugsetlibrary = transposer(side_effect_dict)

In [9]:
# Matching all perturbation id values with drug names #
drugsetlibrary = {k.lower() : [drug_name_conversion.get(x,x) for x in v] for k,v in drugsetlibrary.items()}

In [10]:
len(drugsetlibrary)

985

### Library counts

In [11]:
library_counts(drugsetlibrary)

4444 unique drugs
985 unique association terms
163005 unique associations


### Exporting the drug-set library in GMT format

In [12]:
os.chdir('../data/L1000FWD')

In [13]:
gmt_formatter(drugsetlibrary, 'L1000FWD_predicted_side_effects.txt')